In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from scipy.stats import randint
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import joblib

# **Pre-Processing**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/hospital_data_sampleee.csv', encoding="utf-8-sig")
df.head()

,Date,Medication Revenue,Lab Cost,Consultation Revenue,Doctor Type,Financial Class,Patient Type,Entry Time,Post-Consultation Time,Completion Time,Patient ID
0,11/6/2019,$600.00,$-,$-,ANCHOR,MEDICARE,OUTPATIENT,9:38:34 AM,10:55:50 AM,10:58:02 AM,C10004
1,11/6/2019,$452.70,$-,$-,ANCHOR,MEDICARE,OUTPATIENT,1:46:52 PM,2:37:19 PM,2:51:14 PM,C10014
2,11/5/2019,$444.00,$-,$-,ANCHOR,HMO,OUTPATIENT,10:41:25 AM,11:25:26 AM,11:26:09 AM,C10015
3,11/8/2019,$378.00,$-,$-,ANCHOR,HMO,OUTPATIENT,12:58:50 PM,6:03:12 PM,6:10:58 PM,C10028
4,11/1/2019,$351.00,$10.00,$-,ANCHOR,MEDICARE,OUTPATIENT,10:13:21 AM,11:06:46 AM,11:09:28 AM,C10035


In [ ]:
df.columns = df.columns.str.strip().str.replace("\n","").str.lower().str.replace(" ", "_")

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df["entry_time"] = pd.to_datetime(df["entry_time"])
df["post_consultation_time"] = pd.to_datetime(df["post-consultation_time"])

/tmp/ipython-input-1010585761.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["entry_time"] = pd.to_datetime(df["entry_time"])
/tmp/ipython-input-1010585761.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["post_consultation_time"] = pd.to_datetime(df["post-consultation_time"])


In [ ]:
df["waiting_time"] = (df["post_consultation_time"] - df["entry_time"]).dt.total_seconds() / 60
df = df[df["waiting_time"] >= 0]

In [ ]:
df["entry_hour"] = df["entry_time"].dt.hour
df["entry_dayofweek"] = df["entry_time"].dt.dayofweek
df["entry_minute"] = df["entry_time"].dt.minute

In [ ]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["dayofweek"] = df["date"].dt.dayofweek

In [ ]:
money_cols = ["medication_revenue", "lab_cost", "consultation_revenue"]
for col in money_cols:
    df[col] = pd.to_numeric(df[col].replace('[\$,]', '', regex=True).str.strip(), errors='coerce').fillna(0.0)

<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1899772844.py:3: SyntaxWarning: invalid escape sequence '\$'
  df[col] = pd.to_numeric(df[col].replace('[\$,]', '', regex=True).str.strip(), errors='coerce').fillna(0.0)


In [ ]:
df = df.drop(columns=[
    "patient_id",
    "post_consultation_time",
    "entry_time",
    "completion_time",
    "date"
])

In [ ]:
for col in df.columns:
    if "time" in col and df[col].dtype == "object":
        print("Dropping:", col)
        df = df.drop(columns=[col])

Dropping: post-consultation_time


In [ ]:
X = df.drop(columns=["waiting_time"])
y = df["waiting_time"]

In [ ]:
print(X.dtypes)

medication_revenue      float64
lab_cost                float64
consultation_revenue    float64
doctor_type              object
financial_class          object
patient_type             object
entry_hour                int32
entry_dayofweek           int32
entry_minute              int32
year                      int32
month                     int32
dayofweek                 int32
dtype: object


In [ ]:
categorical_cols = ["doctor_type", "financial_class", "patient_type"]
for col in categorical_cols:
    X[col] = X[col].astype(str)

In [ ]:
numeric_cols = [col for col in X.columns if col not in categorical_cols]

In [ ]:
y_log = np.log1p(y)

In [ ]:
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

# **Random Forest Tree**

In [ ]:
model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("rf", RandomForestRegressor(
        n_estimators=200,
        max_depth=None,
        random_state=42
    ))
])

In [ ]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

In [ ]:
pipeline = Pipeline([
    ("preprocess", preprocess),
    ("regressor", rf)
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

In [ ]:
param_dist = {
    "regressor__n_estimators": randint(100, 500),
    "regressor__max_depth": randint(5, 30),
    "regressor__min_samples_split": randint(2, 10),
    "regressor__min_samples_leaf": randint(1, 5),
    "regressor__max_features": ["sqrt", "log2"]
}

In [ ]:
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42
)

In [ ]:
search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END regressor__max_depth=11, regressor__max_features=log2, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=206; total time=   2.1s
[CV] END regressor__max_depth=11, regressor__max_features=log2, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=206; total time=   2.1s
[CV] END regressor__max_depth=11, regressor__max_features=log2, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=206; total time=   2.0s
[CV] END regressor__max_depth=12, regressor__max_features=sqrt, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=221; total time=   2.6s
[CV] END regressor__max_depth=12, regressor__max_features=sqrt, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=221; total time=   3.8s
[CV] END regressor__max_depth=12, regressor__max_features=sqrt, 

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocess',
                                              ColumnTransformer(transformers=[('cat',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['doctor_type',
                                                                                'financial_class',
                                                                                'patient_type']),
                                                                              ('num',
                                                                               'passthrough',
                                                                               ['medication_revenue',
                                                                                'lab_cost',
                                                                                'consultation_revenue',
                                                                                'entry_hour',
                                                                                'entry_dayofweek',
                                                                                'entry_minute',
                                                                                'year',
                                                                                'month',
                                                                                'dayofweek'])])),
                                             ('r...
                                        'regressor__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f07a7230>,
                                        'regressor__min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f07a73e0>,
                                        'regressor__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f07a7470>},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [ ]:
y_pred_log = search.predict(X_test)
y_pred = np.expm1(y_pred_log)

In [ ]:
mae_rft = mean_absolute_error(np.expm1(y_test), y_pred)
rmse_rft = mean_squared_error(np.expm1(y_test), y_pred)
r2_rft = r2_score(np.expm1(y_test), y_pred)

In [ ]:
print(f"Mean Absolute Error after tuning: {mae_rft:.2f} minutes")
print(f"RMSE: {rmse_rft:.2f} minutes")
print(f"R² Score: {r2_rft:.4f}")

Mean Absolute Error after tuning: 22.53 minutes
RMSE: 1849.22 minutes
R² Score: 0.0429


# **Gradient Boost**

In [ ]:
gbr = GradientBoostingRegressor(random_state=42)

In [ ]:
pipeline_gbr = Pipeline([
    ("preprocess", preprocess),
    ("regressor", gbr)
])

In [ ]:
param_dist_gbr = {
    "regressor__n_estimators": randint(100, 400),
    "regressor__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "regressor__max_depth": randint(2, 6),
    "regressor__min_samples_split": randint(2, 10),
    "regressor__min_samples_leaf": randint(1, 5),
}

In [ ]:
search_gbr = RandomizedSearchCV(
    pipeline_gbr,
    param_distributions=param_dist_gbr,
    n_iter=20,
    cv=3,
    scoring="neg_mean_absolute_error",
    verbose=2,
    random_state=42
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

In [ ]:
search_gbr.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END regressor__learning_rate=0.1, regressor__max_depth=5, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=206; total time=   6.7s
[CV] END regressor__learning_rate=0.1, regressor__max_depth=5, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=206; total time=   5.8s
[CV] END regressor__learning_rate=0.1, regressor__max_depth=5, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=206; total time=   5.9s
[CV] END regressor__learning_rate=0.2, regressor__max_depth=2, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=221; total time=   3.5s
[CV] END regressor__learning_rate=0.2, regressor__max_depth=2, regressor__min_samples_leaf=1, regressor__min_samples_split=8, regressor__n_estimators=221; total time=   2.7s
[CV] END regressor__learning_rate=0.2, regressor__max_depth=2, regres

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocess',
                                              ColumnTransformer(transformers=[('cat',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['doctor_type',
                                                                                'financial_class',
                                                                                'patient_type']),
                                                                              ('num',
                                                                               'passthrough',
                                                                               ['medication_revenue',
                                                                                'lab_cost',
                                                                                'consultation_revenue',
                                                                                'entry_hour',
                                                                                'entry_dayofweek',
                                                                                'entry_minute',
                                                                                'year',
                                                                                'month',
                                                                                'dayofweek'])])),
                                             ('r...
                                        'regressor__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2725fbfa70>,
                                        'regressor__min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f079d850>,
                                        'regressor__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f03617c0>},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [ ]:
y_pred_log_gbr = search_gbr.predict(X_test)
y_pred_gbr = np.expm1(y_pred_log_gbr)

In [ ]:
mae_gbr = mean_absolute_error(np.expm1(y_test), y_pred_gbr)
rmse_gbr = mean_squared_error(np.expm1(y_test), y_pred_gbr)
r2_gbr = r2_score(np.expm1(y_test), y_pred_gbr)

In [ ]:
print(f"MAE: {mae_gbr:.2f} minutes")
print(f"RMSE: {rmse_gbr:.2f} minutes")
print(f"R² Score: {r2_gbr:.4f}")

MAE: 22.35 minutes
RMSE: 1842.53 minutes
R² Score: 0.0463


# **X-Gradient Boost**

In [ ]:
xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

In [ ]:
pipeline_xgb = Pipeline([
    ("preprocess", preprocess),
    ("regressor", xgb)
])

In [ ]:
param_dist_xgb = {
    "regressor__n_estimators": randint(200, 800),
    "regressor__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "regressor__max_depth": randint(3, 10),
    "regressor__subsample": [0.7, 0.8, 1.0],
    "regressor__colsample_bytree": [0.6, 0.8, 1.0],
}

In [ ]:
search_xgb = RandomizedSearchCV(
    pipeline_xgb,
    param_distributions=param_dist_xgb,
    n_iter=20,
    cv=3,
    scoring="neg_mean_absolute_error",
    random_state=42,
    verbose=2
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

In [ ]:
search_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END regressor__colsample_bytree=1.0, regressor__learning_rate=0.2, regressor__max_depth=7, regressor__n_estimators=470, regressor__subsample=1.0; total time=   2.0s
[CV] END regressor__colsample_bytree=1.0, regressor__learning_rate=0.2, regressor__max_depth=7, regressor__n_estimators=470, regressor__subsample=1.0; total time=   1.6s
[CV] END regressor__colsample_bytree=1.0, regressor__learning_rate=0.2, regressor__max_depth=7, regressor__n_estimators=470, regressor__subsample=1.0; total time=   1.4s
[CV] END regressor__colsample_bytree=0.6, regressor__learning_rate=0.01, regressor__max_depth=9, regressor__n_estimators=321, regressor__subsample=1.0; total time=   1.8s
[CV] END regressor__colsample_bytree=0.6, regressor__learning_rate=0.01, regressor__max_depth=9, regressor__n_estimators=321, regressor__subsample=1.0; total time=   1.9s
[CV] END regressor__colsample_bytree=0.6, regressor__learning_rate=0.01, regressor__max

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocess',
                                              ColumnTransformer(transformers=[('cat',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['doctor_type',
                                                                                'financial_class',
                                                                                'patient_type']),
                                                                              ('num',
                                                                               'passthrough',
                                                                               ['medication_revenue',
                                                                                'lab_cost',
                                                                                'consultation_revenue',
                                                                                'entry_hour',
                                                                                'entry_dayofweek',
                                                                                'entry_minute',
                                                                                'year',
                                                                                'month',
                                                                                'dayofweek'])])),
                                             ('r...
                                                                        1.0],
                                        'regressor__learning_rate': [0.01, 0.05,
                                                                     0.1, 0.2],
                                        'regressor__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f0270c50>,
                                        'regressor__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f26f0604680>,
                                        'regressor__subsample': [0.7, 0.8,
                                                                 1.0]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [ ]:
y_pred_log_xgb = search_xgb.predict(X_test)
y_pred_xgb = np.expm1(y_pred_log_xgb)

In [ ]:
mae_xgb = mean_absolute_error(np.expm1(y_test), y_pred_xgb)
rmse_xgb = mean_squared_error(np.expm1(y_test), y_pred_xgb)
r2_xgb = r2_score(np.expm1(y_test), y_pred_xgb)

In [ ]:
print(f"MAE: {mae_xgb:.2f} minutes")
print(f"RMSE: {rmse_xgb:.2f} minutes")
print(f"R² Score: {r2_xgb:.4f}")

MAE: 22.35 minutes
RMSE: 1836.65 minutes
R² Score: 0.0494


# **Artifacts**

In [ ]:
best_model = search_xgb.best_estimator_

In [ ]:
joblib.dump(best_model, "hospital_waiting_model.pkl")

['hospital_waiting_model.pkl']